Quantum Convolutional Neural Network

This is a circuit that requires a connectivity map where the distance between the qubits increases exponentially with depth

In [206]:
from qiskit_ibm_runtime.fake_provider import FakeTorino
from qiskit import QuantumCircuit

backend = FakeTorino()

import time
import numpy as np

Define QCNN circuit

In [207]:
num_qubits = 8
num_layers = int(np.ceil(np.log2(num_qubits)))

qc=QuantumCircuit(num_qubits)
i_conv=0
for i_layer in range(num_layers):
    for i_sub_layer in [0 , 2**i_layer]:            
        for i_q1 in range(i_sub_layer, num_qubits, 2**(i_layer+1)):
            i_q2=2**i_layer+i_q1
            if i_q2<num_qubits:
                qc.rxx(np.random.rand(), i_q1, i_q2)
                qc.ry(np.random.rand(), i_q1)
                qc.ry(np.random.rand(), i_q2)
                i_conv+=1

Optionally print out the circuit

In [208]:
qc.draw()

┌────────────────┐┌─────────────┐                                   »
q_0: ┤0               ├┤ Ry(0.62504) ├───────────────────────────────────»
     │  Rxx(0.088397) │├─────────────┤ ┌───────────────┐ ┌─────────────┐ »
q_1: ┤1               ├┤ Ry(0.96914) ├─┤0              ├─┤ Ry(0.45552) ├─»
     ├───────────────┬┘├─────────────┤ │  Rxx(0.36086) │ ├─────────────┤ »
q_2: ┤0              ├─┤ Ry(0.26047) ├─┤1              ├─┤ Ry(0.95287) ├─»
     │  Rxx(0.78661) │ ├─────────────┤ ├───────────────┤ ├─────────────┴┐»
q_3: ┤1              ├─┤ Ry(0.71231) ├─┤0              ├─┤ Ry(0.011149) ├»
     ├───────────────┤ ├─────────────┤ │  Rxx(0.31125) │┌┴──────────────┤»
q_4: ┤0              ├─┤ Ry(0.96089) ├─┤1              ├┤ Ry(0.0038629) ├»
     │  Rxx(0.49166) │ ├─────────────┤ ├───────────────┤└┬─────────────┬┘»
q_5: ┤1              ├─┤ Ry(0.50247) ├─┤0              ├─┤ Ry(0.94566) ├─»
     └┬──────────────┤ ├─────────────┴┐│  Rxx(0.27126) │ ├─────────────┤ »
q_6: ─┤0             ├─┤ Ry(0.058756) ├┤1              ├─┤ Ry(0.29014) ├─»
      │  Rxx(0.3953) │ ├─────────────┬┘└───────────────┘ └─────────────┘ »
q_7: ─┤1             ├─┤ Ry(0.91105) ├───────────────────────────────────»
      └──────────────┘ └─────────────┘                                   »
«     ┌───────────────┐┌─────────────┐                                »
«q_0: ┤0              ├┤ Ry(0.57102) ├────────────────────────────────»
«     │               │└─────────────┘                                »
«q_1: ┤  Rxx(0.21316) ├───────────────────────────────────────────────»
«     │               │┌─────────────┐┌───────────────┐┌─────────────┐»
«q_2: ┤1              ├┤ Ry(0.01878) ├┤0              ├┤ Ry(0.70477) ├»
«     └───────────────┘└─────────────┘│               │└─────────────┘»
«q_3: ────────────────────────────────┤  Rxx(0.38852) ├───────────────»
«     ┌───────────────┐┌─────────────┐│               │┌─────────────┐»
«q_4: ┤0              ├┤ Ry(0.66663) ├┤1              ├┤ Ry(0.26493) ├»
«     │               │└─────────────┘└───────────────┘└─────────────┘»
«q_5: ┤  Rxx(0.75786) ├───────────────────────────────────────────────»
«     │               │┌─────────────┐                                »
«q_6: ┤1              ├┤ Ry(0.56367) ├────────────────────────────────»
«     └───────────────┘└─────────────┘                                »
«q_7: ────────────────────────────────────────────────────────────────»
«                                                                     »
«     ┌───────────────┐┌─────────────┐
«q_0: ┤0              ├┤ Ry(0.73017) ├
«     │               │└─────────────┘
«q_1: ┤               ├───────────────
«     │               │               
«q_2: ┤  Rxx(0.33549) ├───────────────
«     │               │               
«q_3: ┤               ├───────────────
«     │               │ ┌────────────┐
«q_4: ┤1              ├─┤ Ry(0.4025) ├
«     └───────────────┘ └────────────┘
«q_5: ────────────────────────────────
«                                     
«q_6: ────────────────────────────────
«                                     
«q_7: ────────────────────────────────
«

Compile with default passes

In [209]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
default_pm = generate_preset_pass_manager(backend=backend, optimization_level=3)

t1 = time.time()
default_qc = default_pm.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = default_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("sx", 0) + gate_counts.get("rz", 0) + gate_counts.get("x", 0))

Time taken:  0.11630678176879883
OrderedDict({'rz': 90, 'sx': 90, 'cz': 32, 'x': 10})
Number of 2-qubit gates:  32
Number of 1-qubit gates:  190


Create custom compiler

In [214]:
from qiskit.transpiler import PassManager
import qiskit.transpiler.passes as passes 


custom_pm = PassManager()

custom_pm.append(passes.Collect2qBlocks())
custom_pm.append(passes.ConsolidateBlocks())

# custom_pm.append(passes.Decompose())
# custom_pm.append(passes.Optimize1qGatesDecomposition())

custom_qc = custom_pm.run(qc)

gate_counts = custom_qc.count_ops()
print(gate_counts)

custom_qc.draw()

OrderedDict({'ry': 22, 'rxx': 11})


┌────────────────┐┌─────────────┐                                   »
q_0: ┤0               ├┤ Ry(0.62504) ├───────────────────────────────────»
     │  Rxx(0.088397) │├─────────────┤ ┌───────────────┐ ┌─────────────┐ »
q_1: ┤1               ├┤ Ry(0.96914) ├─┤0              ├─┤ Ry(0.45552) ├─»
     ├───────────────┬┘├─────────────┤ │  Rxx(0.36086) │ ├─────────────┤ »
q_2: ┤0              ├─┤ Ry(0.26047) ├─┤1              ├─┤ Ry(0.95287) ├─»
     │  Rxx(0.78661) │ ├─────────────┤ ├───────────────┤ ├─────────────┴┐»
q_3: ┤1              ├─┤ Ry(0.71231) ├─┤0              ├─┤ Ry(0.011149) ├»
     ├───────────────┤ ├─────────────┤ │  Rxx(0.31125) │┌┴──────────────┤»
q_4: ┤0              ├─┤ Ry(0.96089) ├─┤1              ├┤ Ry(0.0038629) ├»
     │  Rxx(0.49166) │ ├─────────────┤ ├───────────────┤└┬─────────────┬┘»
q_5: ┤1              ├─┤ Ry(0.50247) ├─┤0              ├─┤ Ry(0.94566) ├─»
     └┬──────────────┤ ├─────────────┴┐│  Rxx(0.27126) │ ├─────────────┤ »
q_6: ─┤0             ├─┤ Ry(0.058756) ├┤1              ├─┤ Ry(0.29014) ├─»
      │  Rxx(0.3953) │ ├─────────────┬┘└───────────────┘ └─────────────┘ »
q_7: ─┤1             ├─┤ Ry(0.91105) ├───────────────────────────────────»
      └──────────────┘ └─────────────┘                                   »
«     ┌───────────────┐┌─────────────┐                                »
«q_0: ┤0              ├┤ Ry(0.57102) ├────────────────────────────────»
«     │               │└─────────────┘                                »
«q_1: ┤  Rxx(0.21316) ├───────────────────────────────────────────────»
«     │               │┌─────────────┐┌───────────────┐┌─────────────┐»
«q_2: ┤1              ├┤ Ry(0.01878) ├┤0              ├┤ Ry(0.70477) ├»
«     └───────────────┘└─────────────┘│               │└─────────────┘»
«q_3: ────────────────────────────────┤  Rxx(0.38852) ├───────────────»
«     ┌───────────────┐┌─────────────┐│               │┌─────────────┐»
«q_4: ┤0              ├┤ Ry(0.66663) ├┤1              ├┤ Ry(0.26493) ├»
«     │               │└─────────────┘└───────────────┘└─────────────┘»
«q_5: ┤  Rxx(0.75786) ├───────────────────────────────────────────────»
«     │               │┌─────────────┐                                »
«q_6: ┤1              ├┤ Ry(0.56367) ├────────────────────────────────»
«     └───────────────┘└─────────────┘                                »
«q_7: ────────────────────────────────────────────────────────────────»
«                                                                     »
«     ┌───────────────┐┌─────────────┐
«q_0: ┤0              ├┤ Ry(0.73017) ├
«     │               │└─────────────┘
«q_1: ┤               ├───────────────
«     │               │               
«q_2: ┤  Rxx(0.33549) ├───────────────
«     │               │               
«q_3: ┤               ├───────────────
«     │               │ ┌────────────┐
«q_4: ┤1              ├─┤ Ry(0.4025) ├
«     └───────────────┘ └────────────┘
«q_5: ────────────────────────────────
«                                     
«q_6: ────────────────────────────────
«                                     
«q_7: ────────────────────────────────
«

In [215]:
custom_qc = default_pm.run(custom_qc)

# Get gate counts
gate_counts = custom_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0))
print("Number of 1-qubit gates: ", gate_counts.get("sx", 0) + gate_counts.get("rz", 0) + gate_counts.get("x", 0))


OrderedDict({'sx': 85, 'rz': 82, 'cz': 32, 'x': 10})
Number of 2-qubit gates:  32
Number of 1-qubit gates:  177
